In [62]:
import pandas as pd
import imp
import pprint
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.font_manager import FontProperties
from tqdm.notebook import tqdm
%matplotlib qt
fontP = FontProperties()
fontP.set_size('xx-small')

In [115]:
import eval
imp.reload(eval)


settings = {
        'type': "nhl_SOG",
        'csvs': ["my_g"],
        'sep': ';',
        'bet_on': ['under', 'over'],
        'kelly_risk': 0.1,
        'start': 1,
        'verb': True,
        'kelly_up_lim': 1,
        'kelly_low_lim': 0.1,
        'proba_up_lim': 0.7,
        'proba_low_lim': 0.5,
        'rev_data': False
}
e = eval.daily(settings)
ret = e.eval()


for key, value in ret.items():
        csv = value['settings']['csvs'][0]
        kelly_risk = value['settings']['kelly_risk']

        # Plot
        fig = plt.figure()
        plt.title(f'ROI for \"{csv}.csv\" with risk of: {kelly_risk}')
        plt.xlabel('Potential bets (over and under for each player in each game)')
        plt.ylabel('ROI (if start = 1 else just return)')
        plt.grid()
        plots=[]

        # Plot each day
        plots.append(plt.plot(ret[f'{csv}-{kelly_risk}']['data']['days'], ret[f'{csv}-{kelly_risk}']['data']['days_return'], label = f"{csv}-days")[0])

        # Plot each bet
        x = list(range(0, ret[f'{csv}-{kelly_risk}']['total_bets']))
        plots.append(plt.plot(x, ret[f'{csv}-{kelly_risk}']['data']['bets_return'], label = f"{csv}-bets")[0])

        plt.legend(handles=plots, title='Lines', bbox_to_anchor=(1, 1), loc='upper left', prop=fontP)
        plt.show()

Csv: "my_g.csv" and Pre: 0.1
Ending 2021-04-27 with 103.42% of starting capital
Ending 2021-04-28 with 110.06% of starting capital
Ending 2021-04-29 with 105.35% of starting capital
Ending 2021-05-02 with 104.88% of starting capital
Ending 2021-05-03 with 106.24% of starting capital
Ending 2021-05-04 with 116.88% of starting capital
Ending 2021-05-05 with 119.75% of starting capital
Ending 2021-05-06 with 129.29% of starting capital
Ending 2021-05-07 with 141.68% of starting capital
Ending 2021-05-09 with 140.23% of starting capital
Ending 2021-05-12 with 134.92% of starting capital
Ending 2021-05-17 with 134.93% of starting capital
Ending 2021-05-18 with 146.38% of starting capital
Ending 2021-05-19 with 131.02% of starting capital
Ending 2021-05-20 with 127.43% of starting capital
Ending 2021-05-21 with 135.75% of starting capital
Ending 2021-05-22 with 135.82% of starting capital
Ending 2021-05-23 with 129.38% of starting capital
Ending 2021-05-24 with 114.16% of starting capital
be

In [114]:
import eval
imp.reload(eval)


settings = {
        'type': "CL_HomeXAway",
        'csvs': ["CL"],
        'bet_on': ["home", "draw", "away"],#, "home_draw", "home_away", "home_away"],
        'kelly_risk': 0.2,
        'start': 1,
        'verb': False,
        'kelly_up_lim': 1,
        'kelly_low_lim': 0.1,
        'proba_up_lim': 0.7,
        'proba_low_lim': 0.5,
        'rev_data': False
}
e = eval.daily(settings)
ret = e.eval()

for key, value in ret.items():
        csv = value['settings']['csvs'][0]
        kelly_risk = value['settings']['kelly_risk']

        # Plot
        fig = plt.figure()
        plt.title(f'ROI for \"{csv}.csv\" with risk of: {kelly_risk}')
        plt.xlabel('Potential bets (over and under for each player in each game)')
        plt.ylabel('ROI (if start = 1 else just return)')
        plt.grid()
        plots=[]

        # Plot each day
        plots.append(plt.plot(ret[f'{csv}-{kelly_risk}']['data']['days'], ret[f'{csv}-{kelly_risk}']['data']['days_return'], label = f"{csv}-days")[0])

        # Plot each bet
        x = list(range(0, ret[f'{csv}-{kelly_risk}']['total_bets']))
        plots.append(plt.plot(x, ret[f'{csv}-{kelly_risk}']['data']['bets_return'], label = f"{csv}-bets")[0])

        plt.legend(handles=plots, title='Lines', bbox_to_anchor=(1, 1), loc='upper left', prop=fontP)
        plt.show()

In [119]:
df = pd.read_csv(f"./my_g.csv", sep=';')
df['kelly_under'] = ((df['odds_under'] - 1) * df['proba_under'] - df['proba_over']) / (df['odds_under'] - 1)
df['kelly_over'] = ((df['odds_over'] - 1) * df['proba_over'] - df['proba_under']) / (df['odds_over'] - 1)

kelly_over_lim = 1
kelly_low_lim = 0.1
proba_over_lim = 0.7
proba_low_lim = 0.5

df['bet_under'] = (df['kelly_under'] > kelly_low_lim) & (df['kelly_under'] < kelly_over_lim) \
    & (df['proba_under'] > proba_low_lim) & (df['proba_under'] < proba_over_lim)
df['bet_over'] = (df['kelly_over'] > kelly_low_lim) & (df['kelly_under'] < kelly_over_lim) \
    & (df['proba_over'] > proba_low_lim) & (df['proba_over'] < proba_over_lim)

df['result_under'] = np.where(df['bet_under'], (np.where(df['answer'] == 0, df['kelly_under'], -df['kelly_under'])), 0)
df['result_over'] = np.where(df['bet_over'], (np.where(df['answer'] == 1, df['kelly_over'], -df['kelly_over'])), 0)
df['sum_under'] = df.result_under.cumsum()
df['sum_over'] = df.result_over.cumsum()
df['tot_sum'] = df['sum_under'] + df['sum_over']
df['tot_dif'] = df['sum_over'] - df['sum_under']

df.plot(x="date", y=["sum_under", "sum_over", "tot_sum", "tot_dif"])
plt.grid()


,player_id,gamePk,date,odds_under,odds_over,num_shots,answer,target,pred,proba_under,...,kelly_under,kelly_over,bet_under,bet_over,result_under,result_over,sum_under,sum_over,tot_sum,tot_dif
0,8480002,2020020716,2021-04-27,1.68,2.30,1,0,2.5,0,0.621029,...,0.063719,-0.098744,False,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,8480027,2020020774,2021-04-27,1.72,2.10,4,1,2.5,1,0.448760,...,-0.316852,0.143277,False,True,0.000000,0.143277,0.000000,0.143277,0.143277,0.143277
2,8480036,2020020774,2021-04-27,2.20,1.60,2,1,1.5,1,0.433483,...,-0.038614,-0.155956,False,False,0.000000,0.000000,0.000000,0.143277,0.143277,0.143277
3,8480036,2020020774,2021-04-27,1.66,2.20,2,0,2.5,0,0.589337,...,-0.032880,-0.080451,False,False,0.000000,0.000000,0.000000,0.143277,0.143277,0.143277
4,8480039,2020020774,2021-04-27,2.10,1.70,0,0,1.5,1,0.399945,...,-0.145560,0.028706,False,False,0.000000,0.000000,0.000000,0.143277,0.143277,0.143277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296,8479344,2020030184,2021-05-24,2.65,1.50,2,1,1.5,1,0.482165,...,0.168326,-0.446496,False,False,0.000000,0.000000,-0.367738,0.627446,0.259707,0.995184
1297,8479344,2020030184,2021-05-24,1.57,2.50,2,0,2.5,0,0.596178,...,-0.112282,0.006370,False,False,0.000000,0.000000,-0.367738,0.627446,0.259707,0.995184
1298,8480018,2020030173,2021-05-24,2.30,1.65,3,1,1.5,0,0.512388,...,0.137302,-0.300677,True,False,-0.137302,0.000000,-0.505040,0.627446,0.122405,1.132486
1299,8480145,2020030184,2021-05-24,2.35,1.66,1,0,1.5,1,0.430456,...,0.008572,-0.082663,False,False,0.000000,0.000000,-0.505040,0.627446,0.122405,1.132486
